<a href="https://colab.research.google.com/github/jeswanth2305/5900-project/blob/main/code/SVM_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SVM - Feature Extraction**

**Import Libraries**

Import necessary libraries

In [39]:
import glob
import os
import librosa
import time
import numpy as np
import pandas as pd

**Load all files**

We will create our numpy array extracting Mel-frequency cepstral coefficients (MFCCs) while the classes to predict will be extracted from the name of the file.

**Defining emotions to classify**

Selecting the emotions to be classified. Note that the emotions 'neutral', 'calm' and 'surprised' are only found in RAVDESS dataset and 'pleasantly surprised' or 'ps' is only available in TESS dataset. To combine all the emotions from both datasets into 8 emotion classes, we have changed 'pleasantly surprised' to 'surprised' and 'fearful' to 'fear'

In [40]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprised'
}

#defined tess emotions to test on TESS dataset only
tess_emotions=['angry','disgust','fear','ps','happy','sad']

##defined RAVDESS emotions to test on RAVDESS dataset only
ravdess_emotions=['neutral','calm','angry', 'happy','disgust','sad','fear','surprised']

observed_emotions = ['sad','angry','happy','disgust','surprised','neutral','calm','fear']

**Feature extraction**

Using librosa package we can extract the MFCC features. This function loads the file give the file path and after resampling and computing MFCC features, returns the features. We have selected the no. of MFCCs as 40.

https://librosa.org/librosa/generated/librosa.feature.mfcc.html

In [41]:
def extract_feature(file_name, mfcc):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        return mfccs

**Choosing a dataset**

Choose the dataset(s) you want to load using the following function

In [42]:
def dataset_options():
    # choose datasets
    ravdess = True
    tess = True
    ravdess_speech = False
    ravdess_song = False
    data = {'ravdess':ravdess, 'ravdess_speech':ravdess_speech, 'ravdess_song':ravdess_song, 'tess':tess}
    print(data)
    return data

**Load data**

Load data from the datasets required which is obtained by calling the function dataset__options(). Extract features from each file with the selected emotions in chosen datasets using the extract_feature() function defined.

In [66]:
def load_data(test_size=0.2):
    x,y=[],[]

    # feature to extract
    mfcc = True

    data = dataset_options()

    for file in glob.glob("/content/drive/MyDrive/RAVDESS files/Song and speech/*.wav"):
      file_name = os.path.basename(file)
      parts = file_name.split("-")
      if len(parts) >= 3:
        emotion = emotions[parts[2]]
        if emotion not in observed_emotions:  # options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
            continue
        feature = extract_feature(file, mfcc)
        x.append(feature)
        y.append(emotion)
      else:
        print(f"File name does not have the expected format: {file_name}")

    for file in glob.glob("/content/drive/MyDrive/RAVDESS files/TESS/*.wav"):
      file_name = os.path.basename(file)
      parts = file_name.split("_")
      if len(parts) >= 3:
        emotion = parts[2][:-4]  # split and remove .wav
        if emotion == 'ps':
            emotion = 'surprised'
        if emotion not in observed_emotions:  # options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
            continue
        feature = extract_feature(file, mfcc)
        x.append(feature)
        y.append(emotion)
      else:
        print(f"File name does not have the expected format: {file_name}")
    return {"X":x,"y":y}

In [ ]:
start_time = time.time()

Trial_dict = load_data(test_size = 0.3)

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

In [69]:
X = pd.DataFrame(Trial_dict["X"])
y = pd.DataFrame(Trial_dict["y"])

In [70]:
X.shape, y.shape

((5252, 40), (5252, 1))

In [71]:
#renaming the label column to emotion
y=y.rename(columns= {0: 'emotion'})

In [72]:
#concatinating the attributes and label into a single dataframe
data = pd.concat([X, y], axis =1)

In [73]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-738.960205,55.012432,-22.545742,12.643789,-3.285344,-6.439648,-12.282077,-6.486738,-10.541921,3.362543,...,5.970497,4.215719,5.337021,4.385305,3.625052,1.643892,-0.636432,1.814381,3.166791,neutral
1,-715.542786,71.405403,4.975545,18.101927,8.825036,3.230271,-1.688666,-0.206652,-2.554181,4.168936,...,0.443504,0.990664,0.175228,0.727823,1.855822,0.257207,-1.855294,-1.415936,-0.340105,neutral
2,-616.739807,59.600132,-5.550658,13.469154,1.485377,0.201867,-14.613069,-4.188658,-5.495050,-6.319356,...,-2.013380,-1.005495,0.310675,0.595130,-1.666019,-2.078755,-2.563148,-0.593154,2.019405,neutral
3,-698.304199,46.464561,-10.771418,12.419950,-7.534273,-1.659831,-9.686534,-9.666937,-10.174019,-0.902112,...,0.067040,-1.443540,0.970879,2.591219,1.850196,1.659194,0.163973,4.402706,4.185752,neutral
4,-652.819153,57.520428,-12.105348,11.378452,-7.287295,-0.952046,-8.074774,-6.016807,-6.434291,1.423135,...,2.583748,2.572970,3.680408,2.838020,2.145133,-0.954625,0.745138,-0.201660,0.126857,neutral


**Shuffling data**

In [74]:
#reindexing to shuffle the data at random
data = data.reindex(np.random.permutation(data.index))

In [77]:
# Storing shuffled ravdess and tess data to avoid loading again
data.to_csv("RAVTESS_MFCC_Observed.csv")
save_path = '/content/drive/MyDrive/5900 project/RAVTESS_MFCC_Observed.csv'
data.to_csv(save_path)